In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/research'

In [2]:
os.chdir('../')
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    competition_name: str
    train_dataset_filename: str
    test_dataset_filename: str
    train_dataset_local_path: Path
    test_dataset_local_path: Path

In [4]:
from src.cellseg.constant import *
from src.cellseg.utils.main_utils import create_directories, read_yaml

class ConfigurationMananger:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            competition_name=config.competition_name,
            train_dataset_filename=config.train_dataset_filename,
            test_dataset_filename=config.test_dataset_filename,
            train_dataset_local_path = os.path.join(
                config.root_dir,
                config.train_dataset_filename
            ),
            test_dataset_local_path = os.path.join(
                config.root_dir,
                config.test_dataset_filename
            )
        )
        
        return data_ingestion_config

In [6]:
import os
from src.cellseg.utils.main_utils import get_size, create_directories, zip_file_extraction
from src.cellseg import logger

import subprocess

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_kaggle_dataset(self, local_path, competition_name, name):
        if not os.path.exists(local_path):
            info = subprocess.run(
                f"kaggle competitions download -c {competition_name} -f {name} -p {self.config.root_dir}",
                shell=True
            )
            logger.info(f"{name} downloaded with folowing info: \n{info}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(local_path))}")
    
    def download_dataset(self):
        self.download_kaggle_dataset(
            self.config.train_dataset_local_path,
            self.config.competition_name,
            self.config.train_dataset_filename
        )
        
        self.download_kaggle_dataset(
            self.config.test_dataset_local_path,
            self.config.competition_name,
            self.config.test_dataset_filename
        )
    
    def extract_zip_files(self):
        train_unzip_path = os.path.join(self.config.root_dir, 'train')
        test_unzip_path = self.config.root_dir + '/test'
        
        create_directories([train_unzip_path, test_unzip_path])
        
        zip_file_extraction(self.config.train_dataset_local_path, train_unzip_path)
        zip_file_extraction(self.config.test_dataset_local_path, test_unzip_path)
    
    def ingestion_compose(self):
        if not os.path.exists(os.path.join(self.config.root_dir, 'train')) and not os.path.exists(os.path.join(self.config.root_dir, 'test')):
            self.download_dataset()
            self.extract_zip_files()
        else:
            logger.info("Dataset files already exist!")

In [7]:
try:
    config = ConfigurationMananger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.ingestion_compose()

except Exception as e:
    raise e

[2024-11-13 21:15:37,591: INFO: main_utils: created directory at: artifacts]
[2024-11-13 21:15:37,593: INFO: main_utils: created directory at: artifacts/data_ingestion]
[2024-11-13 21:15:37,594: INFO: 764276990: Dataset files already exist!]
